In [2]:
import tushare as ts
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
import ffn
from sklearn.svm import SVR
import matplotlib.pyplot as plt

In [23]:
class Model:
    
    #初始化参数
    def __init__(self, data, train_start, train_end, test_start, test_end, n):
        
        self.code = data['code']
        self.train_start = train_start
        self.train_end = train_end
        self.test_start = test_start
        self.test_end = test_end
        self.n = n
        
    #获取单只股票初始训练集和测试集
    def getInitData(self, code):
        
        train_start = self.train_start
        train_end = self.train_end
        
        test_start = self.test_start
        test_end = self.test_end
        
        train_data = ts.get_hist_data(code, start = train_start, end = train_end)
        test_data = ts.get_hist_data(code, start = test_start, end = test_end)

        train_data = train_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
        test_data = test_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
        
        return train_data, test_data
    
    #训练和预测股票的N期收益
    def train(self):
        #使用scikit-learn导入SVR进行训练和预测
        x_train = []
        y_label = []
        x_test = []
        
        for code in self.code:
            train_data, test_data = self.getInitData(code)
            
            #计算特征
            '''
            以后特征的计算，写在feature的class里即可
            '''
            feature = Feature(train_data, test_data, self.n)
            train_data, test_data = feature.cal_mean()
            train_data = feature.cal_nReturn(train_data)

            svr_rbf = SVR(kernel = 'rbf', C = 1e3, gamma = 0.1)
            X = train_data.iloc[:,-9:-2]
            X1 = test_data.iloc[:,-8:-1]
            y = train_data.iloc[:,-1:]
            for i in range(len(X)):
                x_train.append(list(X.iloc[i]))
            for i in range(len(y)):
                y_label.append(list(y.iloc[i]))
            for i in range(len(X1)):    
                x_test.append(list(X1.iloc[i]))

        y_rbf = svr_rbf.fit(x_train,y_label).predict(x_test)
        
        return y_rbf

    #得到预期代码

#     def getCode():
        
#         return 
        

In [19]:
class Feature:
    
    def __init__(self, train_data, test_data, n):
        self.train_data = train_data
        self.test_data = test_data
        self.n = n
        
    #计算开盘到收盘之间的价格均值
    def cal_mean(self):
        train_data = self.train_data
        test_data = self.test_data
        
        first_data_open = train_data.loc[:,['open']]
        first_data_close = train_data.loc[:,['close']]

        second_data_open = test_data.loc[:,['open']]
        second_data_close = test_data.loc[:,['close']]

        first_data_open.rename(columns = {'open':'mean'},inplace=True)
        first_data_close.rename(columns = {'close':'mean'},inplace=True)
        first_data_mean = (first_data_close + first_data_open)/2

        second_data_open.rename(columns = {'open':'mean'},inplace=True)
        second_data_close.rename(columns = {'close':'mean'},inplace=True)
        second_data_mean = (second_data_close + second_data_open)/2

        train_data['mean'] = first_data_mean
        test_data['mean'] = second_data_mean
        
        return train_data,test_data
    
    #计算N期收益
    def cal_nReturn(self, train_data):
        n = self.n
        #训练集的N期平均收益率
        origin_mean = train_data.loc[:,['mean']]
        origin_mean_n = origin_mean.shift(n)
        train_simpleret = (origin_mean - origin_mean_n) / origin_mean_n
        train_data['n_simpleret'] = train_simpleret
        #填充缺失的值
        train_data = train_data.fillna(method='bfill')
        
        return train_data

In [24]:
#主函数
model = Model(ts.get_sz50s(), '2017-03-01', '2017-06-30', '2017-07-01', '2017-08-31', 15)

print (model.train())

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[-0.01226435  0.00128179  0.00536256 ...,  0.05089279  0.04711263
  0.04966329]
